In [1]:
!pip install mido pretty_midi numpy tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=953a1e11ece42faa6bfa03be85c339a44970f97c6b19983f12004468a18438fd
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pretty_midi
import mido
from mido import MidiFile

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Super Mario 64 - Medley.mid to Super Mario 64 - Medley.mid


In [4]:
def midi_to_sequence(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            notes.append((note.pitch, note.start, note.end))
    return notes

midi_file_path = '/content/Super Mario 64 - Medley.mid'
notes = midi_to_sequence(midi_file_path)

In [5]:
def encode_notes(notes, pitch_range=128):
    encoded = []
    for pitch, start, end in notes:
        encoded.append(pitch)
    return np.array(encoded)

encoded_notes = encode_notes(notes)

In [6]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LENGTH = 50
X, y = create_sequences(encoded_notes, SEQ_LENGTH)

In [7]:
model = Sequential([
    LSTM(128, input_shape=(SEQ_LENGTH, 1), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dense(128, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
X = np.expand_dims(X, axis=-1)

In [17]:
model.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 15s 181ms/step - accuracy: 0.7269 - loss: 0.8711
Epoch 2/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 17s 199ms/step - accuracy: 0.7168 - loss: 0.9081
Epoch 3/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 15s 179ms/step - accuracy: 0.7234 - loss: 0.8871
Epoch 4/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 21s 180ms/step - accuracy: 0.7502 - loss: 0.8366
Epoch 5/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 22s 195ms/step - accuracy: 0.7496 - loss: 0.8199
Epoch 6/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 19s 180ms/step - accuracy: 0.7334 - loss: 0.8584
Epoch 7/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 21s 187ms/step - accuracy: 0.7441 - loss: 0.8176
Epoch 8/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 180ms/step - accuracy: 0.7599 - loss: 0.8004
Epoch 9/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 22s 194ms/step - accuracy: 0.7672 - loss: 0.7911
Epoch 10/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 15s 181ms/step - accuracy: 0.7650 - loss: 0.7864
Epoch 11/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 179ms/step - accuracy: 0.7387 - loss: 0.8278
Epoch 12/20
85/85 ━━━━━━━━━━━━━━━━━━━━ 16

In [18]:
def generate_sequence(model, seed_sequence, length):
    generated = []
    current_seq = seed_sequence.copy()
    for _ in range(length):
        prediction = model.predict(np.expand_dims(np.expand_dims(current_seq, axis=0), axis=-1), verbose=0)
        next_note = np.argmax(prediction)
        generated.append(next_note)
        current_seq = np.append(current_seq[1:], [next_note])
        current_seq = current_seq.reshape(seed_sequence.shape)
    return generated

seed_sequence = X[0]
generated_notes = generate_sequence(model, seed_sequence, length=100)

In [19]:
def sequence_to_midi(sequence, output_file='output.mid'):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    start_time = 0
    for pitch in sequence:
        note = pretty_midi.Note(velocity=100, pitch=pitch, start=start_time, end=start_time + 0.5)
        instrument.notes.append(note)
        start_time += 0.5
    midi.instruments.append(instrument)
    midi.write(output_file)

sequence_to_midi(generated_notes)